In [172]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [173]:
df = pd.read_csv('/Users/thamer-gahtani/Downloads/iron_hack /dh_etas.csv')
df.head()

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_couriers,total_busy_couriers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration
0,1.0,2015-01-21 17:02:39,2015-01-21 18:01:30,5919,american,1.0,3,950,3,350,900,3.0,2.0,1.0,446,628.0
1,1.0,2015-02-12 23:47:42,2015-02-13 00:13:52,3954,vietnamese,3.0,2,1800,2,600,650,4.0,3.0,2.0,446,167.0
2,1.0,2015-02-12 23:52:34,2015-02-13 00:23:19,5654,sandwich,5.0,1,1295,1,1295,1295,6.0,2.0,1.0,251,586.0
3,1.0,2015-02-12 23:52:36,2015-02-13 00:38:47,5560,japanese,3.0,1,2802,1,2387,2398,12.0,16.0,27.0,251,1058.0
4,1.0,2015-02-12 23:35:28,2015-02-13 00:29:22,2406,cafe,4.0,6,7160,4,795,1590,10.0,10.0,10.0,446,536.0


In [174]:
df = df.dropna()

In [175]:
df.shape

(137484, 16)

In [176]:


def hash_trick(series, num_buckets):
    return series.apply(lambda x: hash(x) % num_buckets)

# Apply hash trick to 'store_id' and 'store_primary_category'
num_buckets = 10  # Define the number of hash buckets
df['store_id_hashed'] = hash_trick(df['store_id'], num_buckets)
df['store_primary_category_hashed'] = hash_trick(df['store_primary_category'], num_buckets)

# Drop the original 'store_id' and 'store_primary_category' columns
df = df.drop(columns=['store_id', 'store_primary_category'])

# One-hot encode the remaining columns
df_encoded = pd.get_dummies(df, columns=['market_id', 'order_protocol'])

# Display the first few rows of the encoded DataFrame
df_encoded.head()

,created_at,actual_delivery_time,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_couriers,total_busy_couriers,total_outstanding_orders,...,market_id_4.0,market_id_5.0,market_id_6.0,order_protocol_1.0,order_protocol_2.0,order_protocol_3.0,order_protocol_4.0,order_protocol_5.0,order_protocol_6.0,order_protocol_7.0
0,2015-01-21 17:02:39,2015-01-21 18:01:30,3,950,3,350,900,3.0,2.0,1.0,...,False,False,False,True,False,False,False,False,False,False
1,2015-02-12 23:47:42,2015-02-13 00:13:52,2,1800,2,600,650,4.0,3.0,2.0,...,False,False,False,False,False,True,False,False,False,False
2,2015-02-12 23:52:34,2015-02-13 00:23:19,1,1295,1,1295,1295,6.0,2.0,1.0,...,False,False,False,False,False,False,False,True,False,False
3,2015-02-12 23:52:36,2015-02-13 00:38:47,1,2802,1,2387,2398,12.0,16.0,27.0,...,False,False,False,False,False,True,False,False,False,False
4,2015-02-12 23:35:28,2015-02-13 00:29:22,6,7160,4,795,1590,10.0,10.0,10.0,...,False,False,False,False,False,False,True,False,False,False


In [177]:
df.shape

(137484, 16)

In [178]:
df['created_at'] = pd.to_datetime(df['created_at'])
df['actual_delivery_time'] = pd.to_datetime(df['actual_delivery_time'])

# Extract components from 'created_at'
df['created_year'] = df['created_at'].dt.year
df['created_month'] = df['created_at'].dt.month
df['created_day'] = df['created_at'].dt.day
df['created_hour'] = df['created_at'].dt.hour
df['created_minute'] = df['created_at'].dt.minute
df['created_second'] = df['created_at'].dt.second
df['time_of_day'] = pd.cut(df['created_hour'],
                              bins=[-1, 6, 12, 18, 23],
                              labels=['night', 'morning', 'afternoon', 'evening'])



# Calculate ETA in seconds
df['eta_seconds'] = (df['actual_delivery_time'] - df['created_at']).dt.total_seconds()

In [179]:
df.shape

(137484, 24)

In [180]:
df.head()

,market_id,created_at,actual_delivery_time,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_couriers,...,store_id_hashed,store_primary_category_hashed,created_year,created_month,created_day,created_hour,created_minute,created_second,time_of_day,eta_seconds
0,1.0,2015-01-21 17:02:39,2015-01-21 18:01:30,1.0,3,950,3,350,900,3.0,...,9,2,2015,1,21,17,2,39,afternoon,3531.0
1,1.0,2015-02-12 23:47:42,2015-02-13 00:13:52,3.0,2,1800,2,600,650,4.0,...,4,3,2015,2,12,23,47,42,evening,1570.0
2,1.0,2015-02-12 23:52:34,2015-02-13 00:23:19,5.0,1,1295,1,1295,1295,6.0,...,4,2,2015,2,12,23,52,34,evening,1845.0
3,1.0,2015-02-12 23:52:36,2015-02-13 00:38:47,3.0,1,2802,1,2387,2398,12.0,...,0,1,2015,2,12,23,52,36,evening,2771.0
4,1.0,2015-02-12 23:35:28,2015-02-13 00:29:22,4.0,6,7160,4,795,1590,10.0,...,6,5,2015,2,12,23,35,28,evening,3234.0


In [181]:
df = df.drop(columns=['created_at', 'actual_delivery_time'])
df.head()




df.shape

(137484, 22)

In [182]:
from sklearn.model_selection import TimeSeriesSplit


In [183]:
df_encoded['eta_seconds'] = df['eta_seconds']

# Prepare features and target
X = df_encoded.drop(columns=['eta_seconds'])
y = df_encoded['eta_seconds']

# Initialize TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

# Perform cross-validation
for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [184]:
# ... existing code ...

# Ensure 'eta_seconds' is included in df_encoded
df_encoded['eta_seconds'] = df['eta_seconds']

# Check and convert data types
for column in df_encoded.columns:
    if df_encoded[column].dtype == 'object':
        df_encoded[column] = df_encoded[column].astype('category').cat.codes

# Prepare features and target
X = df_encoded.drop(columns=['eta_seconds'])
y = df_encoded['eta_seconds']

# Initialize TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

# Lists to store metrics for each fold
rmse_list = []
mae_list = []

# Perform cross-validation
for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize and train the XGBoost model
    model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)
    model.fit(X_train, y_train)
    
    # Make predictions
    predictions = model.predict(X_test)
    
    # Evaluate the model
    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    
    # Convert metrics to minutes
    rmse_minutes = np.sqrt(mse) / 60
    mae_minutes = mae / 60
    
    # Store metrics
    rmse_list.append(rmse_minutes)
    mae_list.append(mae_minutes)
    
    print(f'Fold RMSE (minutes): {rmse_minutes}')
    print(f'Fold MAE (minutes): {mae_minutes}')

# Calculate overall metrics
overall_rmse = np.mean(rmse_list)
overall_mae = np.mean(mae_list)

print(f'Overall RMSE (minutes): {overall_rmse}')
print(f'Overall MAE (minutes): {overall_mae}')

# ... existing code ...

Fold RMSE (minutes): 15.936179693037833
Fold MAE (minutes): 11.143208968116005
Fold RMSE (minutes): 18.053850105109696
Fold MAE (minutes): 12.08747193822966
Fold RMSE (minutes): 17.347490285827078
Fold MAE (minutes): 11.199482321241842
Fold RMSE (minutes): 43.12233294214034
Fold MAE (minutes): 10.10803232387496
Fold RMSE (minutes): 16.366438249282503
Fold MAE (minutes): 10.178631159662073
Overall RMSE (minutes): 22.165258255079493
Overall MAE (minutes): 10.943365342224908


In [194]:
model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [189]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

# First, load the data and check its structure
df = pd.read_csv('/Users/thamer-gahtani/Downloads/iron_hack /real_data_etas.csv')
print("Original DataFrame columns:", df.columns.tolist())
print("\nOriginal DataFrame info:")
print(df.info())

def process_datetime(df):
    df = df.copy()
    try:
        df['created_at'] = pd.to_datetime(df['created_at'])
        df['actual_delivery_time'] = pd.to_datetime(df['actual_delivery_time'])
        
        df['created_year'] = df['created_at'].dt.year
        df['created_month'] = df['created_at'].dt.month
        df['created_day'] = df['created_at'].dt.day
        df['created_hour'] = df['created_at'].dt.hour
        df['created_minute'] = df['created_at'].dt.minute
        df['created_second'] = df['created_at'].dt.second
        df['time_of_day'] = pd.cut(df['created_hour'],
                                   bins=[-1, 6, 12, 18, 23],
                                   labels=['night', 'morning', 'afternoon', 'evening']).astype(str)
        
        df['eta_seconds'] = (df['actual_delivery_time'] - df['created_at']).dt.total_seconds()
    except Exception as e:
        print(f"Error in datetime processing: {str(e)}")
        raise
    return df

# After seeing the actual columns, define the correct column lists
numeric_columns = [col for col in df.columns if df[col].dtype in ['int64', 'float64']]
categorical_columns = [col for col in df.columns if df[col].dtype == 'object']

print("\nNumeric columns:", numeric_columns)
print("Categorical columns:", categorical_columns)

# Define the pipeline
pipeline = Pipeline(steps=[
    ('datetime_processing', FunctionTransformer(process_datetime, validate=False)),
    ('impute', ColumnTransformer(
        transformers=[
            ('num', SimpleImputer(strategy='median'), numeric_columns),
            ('cat', SimpleImputer(strategy='most_frequent'), categorical_columns)
        ],
        remainder='passthrough'
    ))
])

# Apply the pipeline
try:
    df_processed = pipeline.fit_transform(df)
    
    # Get all column names after processing
    processed_columns = (
        numeric_columns +
        categorical_columns +
        ['created_year', 'created_month', 'created_day',
         'created_hour', 'created_minute', 'created_second',
         'time_of_day', 'eta_seconds']
    )
    
    # Convert to DataFrame with all columns
    df_processed = pd.DataFrame(df_processed, columns=processed_columns)
    
    # Display results
    print("\nProcessed DataFrame Info:")
    print(df_processed.info())
    print("\nFirst few rows of processed DataFrame:")
    print(df_processed.head())
    print("\nBasic statistics of numeric columns:")
    print(df_processed.describe())
    
except Exception as e:
    print(f"Error during processing: {str(e)}")




Original DataFrame columns: ['market_id', 'created_at', 'store_id', 'store_primary_category', 'order_protocol', 'total_items', 'subtotal', 'num_distinct_items', 'min_item_price', 'max_item_price', 'total_onshift_couriers', 'total_busy_couriers', 'total_outstanding_orders', 'estimated_order_place_duration', 'estimated_store_to_consumer_driving_duration']

Original DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43477 entries, 0 to 43476
Data columns (total 15 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   market_id                                     43250 non-null  float64
 1   created_at                                    43477 non-null  object 
 2   store_id                                      43477 non-null  int64  
 3   store_primary_category                        42444 non-null  object 
 4   order_protocol                                43254 no

In [191]:
df.shape

(43477, 15)

In [192]:
pd.read_csv('/Users/thamer-gahtani/Downloads/iron_hack /real_data_etas.csv').shape

(43477, 15)

In [193]:
process_datetime(df)

Error in datetime processing: 'actual_delivery_time'


KeyError: 'actual_delivery_time'

In [210]:
import pandas as pd
import numpy as np
from datetime import datetime

# Load the new real-life data
new_data = pd.read_csv('/Users/thamer-gahtani/Downloads/iron_hack /real_data_etas.csv')

# Function to hash categorical variables
def hash_trick(series, num_buckets):
    return series.apply(lambda x: hash(x) % num_buckets)

# Process the data to match training data structure
def preprocess_for_prediction(df):
    # Create a copy to avoid modifying the original
    df = df.copy()
    
    # Convert datetime
    df['created_at'] = pd.to_datetime(df['created_at'])
    # Add placeholder for actual_delivery_time
    df['actual_delivery_time'] = df['created_at']
    
    # Apply hash trick to categorical columns
    num_buckets = 10
    df['store_id_hashed'] = hash_trick(df['store_id'], num_buckets)
    df['store_primary_category_hashed'] = hash_trick(df['store_primary_category'], num_buckets)
    
    # One-hot encode market_id and order_protocol
    df_encoded = pd.get_dummies(df, 
                              columns=['market_id', 'order_protocol'],
                              prefix=['market_id', 'order_protocol'])
    
    # Ensure all expected columns exist and in correct order
    feature_columns = [
        'created_at', 'actual_delivery_time', 'total_items', 'subtotal', 
        'num_distinct_items', 'min_item_price', 'max_item_price', 
        'total_onshift_couriers', 'total_busy_couriers', 'total_outstanding_orders',
        'estimated_order_place_duration', 'estimated_store_to_consumer_driving_duration',
        'store_id_hashed', 'store_primary_category_hashed'
    ]
    
    # Add market_id columns in order
    feature_columns.extend([f'market_id_{i}.0' for i in range(1, 7)])
    
    # Add order_protocol columns in order
    feature_columns.extend([f'order_protocol_{i}.0' for i in range(1, 8)])
    
    # Ensure all columns exist
    for col in feature_columns:
        if col not in df_encoded.columns and col.startswith(('market_id_', 'order_protocol_')):
            df_encoded[col] = False
    
    # Convert datetime columns to integers (timestamps)
    df_encoded['created_at'] = df_encoded['created_at'].astype(np.int64) // 10**9
    df_encoded['actual_delivery_time'] = df_encoded['actual_delivery_time'].astype(np.int64) // 10**9
    
    return df_encoded[feature_columns]

# Preprocess the new data
X_predict = preprocess_for_prediction(new_data)

# Store original created_at before prediction
original_created_at = pd.to_datetime(new_data['created_at'])

# Make predictions
predictions = model.predict(X_predict)

# Create results DataFrame
results_df = pd.DataFrame({
    'created_at': original_created_at,
    'predicted_eta_seconds': predictions,
    'predicted_eta_minutes': predictions / 60,
    'predicted_delivery_time': original_created_at + pd.to_timedelta(predictions, unit='s')
})

# Export results
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_filename = f'delivery_predictions_{timestamp}.csv'
results_df.to_csv(output_filename, index=False)
print(f"\nResults exported to {output_filename}")

# Display sample results
print("\nSample of predictions (first 5 rows):")
print(results_df.head())

# Display basic statistics of predictions
print("\nPrediction Statistics (in minutes):")
print(results_df['predicted_eta_minutes'].describe())


Results exported to delivery_predictions_20241129_172304.csv

Sample of predictions (first 5 rows):
           created_at  predicted_eta_seconds  predicted_eta_minutes  \
0 2015-02-13 00:00:02            2860.124023              47.668736   
1 2015-02-13 00:00:04            2946.895752              49.114929   
2 2015-02-13 00:00:18            3128.277588              52.137959   
3 2015-02-13 00:00:50            2499.443848              41.657398   
4 2015-02-13 00:00:55            2568.609131              42.810154   

        predicted_delivery_time  
0 2015-02-13 00:47:42.124023438  
1 2015-02-13 00:49:10.895751953  
2 2015-02-13 00:52:26.277587891  
3 2015-02-13 00:42:29.443847656  
4 2015-02-13 00:43:43.609130859  

Prediction Statistics (in minutes):
count    43477.000000
mean        61.537613
std         20.600466
min         24.972790
25%         46.586288
50%         55.602612
75%         73.585945
max       1053.270386
Name: predicted_eta_minutes, dtype: float64


In [213]:
results_df['predicted_eta_seconds'].to_csv('predicted_etas.csv')

In [217]:
pd.read_csv('predicted_etas.csv').drop(columns=['Unnamed: 0']).to_csv('predicted_etas_clean.csv',index=False)

In [222]:
pd.read_csv('predicted_etas.csv').drop(columns=['Unnamed: 0'],inplace=True)

In [223]:
iiii = pd.read_csv('predicted_etas_clean.csv')
iiii.head()

,predicted_eta_seconds
0,2860.1240
1,2946.8958
2,3128.2776
3,2499.4438
4,2568.6091
